In [32]:
import pandas as pd
import numpy as np
from scipy import stats

from unified_data_provider import get_data

df = get_data("UEFA Champions League 2016-2022 Data 3.xlsx")

df.to_csv("ucl_matches_parsed.csv", index=False)


df.head()


,GOAL_ID,MATCH_ID,PID,DURATION,ASSIST,GOAL_DESC,SEASON,DATE_TIME,HOME_TEAM,AWAY_TEAM,...,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FOOT,IS_PENALTY,FIRST_HALF_GOAL,PLAYER_AGE,TOTAL_MATCH_GOALS,HOME_GOAL
0,gl470,mt154,ply1479,4,ply864,right-footed shot,2020-2021,2020-11-03 20:00:00,FC Porto,Olympique Marseille,...,17.0,Arabia,183.0,83.0,R,0,1,29.557837,3,0
1,gl471,mt154,ply1075,28,ply864,penalty,2020-2021,2020-11-03 20:00:00,FC Porto,Olympique Marseille,...,27.0,Midfielder,181.0,78.0,R,1,1,28.421629,3,0
2,gl472,mt154,ply585,69,ply864,right-footed shot,2020-2021,2020-11-03 20:00:00,FC Porto,Olympique Marseille,...,23.0,Forward,180.0,73.0,R,0,0,23.805613,3,0
3,gl473,mt155,ply396,12,ply653,right-footed shot,2020-2021,2020-11-03 20:00:00,Manchester City,Olympiakos Piraeus,...,11.0,Forward,184.0,77.0,R,0,1,20.678987,3,0
4,gl474,mt155,ply1064,81,ply653,right-footed shot,2020-2021,2020-11-03 20:00:00,Manchester City,Olympiakos Piraeus,...,9.0,Forward,175.0,68.0,R,0,0,23.586585,3,0


In [33]:
categorical_cols = ["FOOT", "POSITION"]

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

df_encoded.head()

,GOAL_ID,MATCH_ID,PID,DURATION,ASSIST,GOAL_DESC,SEASON,DATE_TIME,HOME_TEAM,AWAY_TEAM,...,TOTAL_MATCH_GOALS,HOME_GOAL,FOOT_R,POSITION_Defender,POSITION_Emirates,POSITION_Forward,POSITION_Goalkeeper,POSITION_Midfielder,POSITION_Republic,POSITION_Rica
0,gl470,mt154,ply1479,4,ply864,right-footed shot,2020-2021,2020-11-03 20:00:00,FC Porto,Olympique Marseille,...,3,0,True,False,False,False,False,False,False,False
1,gl471,mt154,ply1075,28,ply864,penalty,2020-2021,2020-11-03 20:00:00,FC Porto,Olympique Marseille,...,3,0,True,False,False,False,False,True,False,False
2,gl472,mt154,ply585,69,ply864,right-footed shot,2020-2021,2020-11-03 20:00:00,FC Porto,Olympique Marseille,...,3,0,True,False,False,True,False,False,False,False
3,gl473,mt155,ply396,12,ply653,right-footed shot,2020-2021,2020-11-03 20:00:00,Manchester City,Olympiakos Piraeus,...,3,0,True,False,False,True,False,False,False,False
4,gl474,mt155,ply1064,81,ply653,right-footed shot,2020-2021,2020-11-03 20:00:00,Manchester City,Olympiakos Piraeus,...,3,0,True,False,False,True,False,False,False,False


In [34]:
rows, cols = df_encoded.shape
rows, cols


(2279, 37)

In [35]:
numeric_cols = df_encoded.select_dtypes(include=["number"]).columns
numeric_cols


Index(['DURATION', 'HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE', 'PENALTY_SHOOT_OUT',
       'ATTENDANCE', 'JERSEY_NUMBER', 'HEIGHT', 'WEIGHT', 'IS_PENALTY',
       'FIRST_HALF_GOAL', 'PLAYER_AGE', 'TOTAL_MATCH_GOALS', 'HOME_GOAL'],
      dtype='object')

In [36]:
numeric_summary = []

for col in numeric_cols:
    s = df_encoded[col]
    numeric_summary.append({
        "column": col,
        "missing_ratio": s.isna().mean(),
        "min": s.min(),
        "max": s.max(),
        "mean": s.mean(),
        "median": s.median(),
        "variance": s.var(),
        "q_0.1": s.quantile(0.1),
        "q_0.9": s.quantile(0.9),
        "q1": s.quantile(0.25),
        "q3": s.quantile(0.75),
    })

numeric_summary_df = pd.DataFrame(numeric_summary)
numeric_summary_df


,column,missing_ratio,min,max,mean,median,variance,q_0.1,q_0.9,q1,q3
0,DURATION,0.000000,0.00000,120.000000,50.306713,52.000000,7.164059e+02,12.000000,87.000000,28.000000,74.00000
1,HOME_TEAM_SCORE,0.000000,0.00000,8.000000,2.235630,2.000000,2.792128e+00,0.000000,5.000000,1.000000,3.00000
2,AWAY_TEAM_SCORE,0.000000,0.00000,8.000000,1.826240,2.000000,2.351708e+00,0.000000,4.000000,1.000000,3.00000
3,PENALTY_SHOOT_OUT,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000
4,ATTENDANCE,0.000000,0.00000,98299.000000,36133.788065,38834.000000,5.903025e+08,0.000000,68145.000000,16324.000000,52628.00000
5,JERSEY_NUMBER,0.071523,1.00000,99.000000,17.632325,11.000000,2.907744e+02,7.000000,30.000000,9.000000,21.00000
6,HEIGHT,0.045195,163.00000,199.000000,180.799173,181.000000,4.395551e+01,172.000000,189.000000,175.000000,185.00000
7,WEIGHT,0.045195,54.00000,101.000000,75.592831,75.000000,4.534494e+01,68.000000,84.000000,70.000000,81.00000
8,IS_PENALTY,0.000000,0.00000,1.000000,0.097850,0.000000,8.831408e-02,0.000000,0.000000,0.000000,0.00000
9,FIRST_HALF_GOAL,0.000000,0.00000,1.000000,0.447126,0.000000,2.473129e-01,0.000000,1.000000,0.000000,1.00000


In [37]:
categorical_cols_original = df.select_dtypes(include=["object"]).columns
categorical_cols_original


Index(['GOAL_ID', 'MATCH_ID', 'PID', 'ASSIST', 'GOAL_DESC', 'SEASON',
       'HOME_TEAM', 'AWAY_TEAM', 'STADIUM', 'PLAYER_ID', 'FIRST_NAME',
       'LAST_NAME', 'NATIONALITY', 'TEAM', 'POSITION', 'FOOT'],
      dtype='object')

In [38]:
categorical_summary = []

for col in categorical_cols_original:
    s = df[col]
    categorical_summary.append({
        "column": col,
        "missing_ratio": s.isna().mean(),
        "unique_values": s.nunique(),
        "mode": s.mode().iloc[0] if not s.mode().empty else None
    })

categorical_summary_df = pd.DataFrame(categorical_summary)
categorical_summary_df


,column,missing_ratio,unique_values,mode
0,GOAL_ID,0.000000,2279,gl1
1,MATCH_ID,0.000000,702,mt688
2,PID,0.045195,666,ply398
3,ASSIST,0.295744,594,ply741
4,GOAL_DESC,0.007021,9,right-footed shot
5,SEASON,0.000000,6,2017-2018
6,HOME_TEAM,0.000000,74,Real Madrid
7,AWAY_TEAM,0.000000,74,Bayern München
8,STADIUM,0.000000,80,Allianz Arena
9,PLAYER_ID,0.045195,666,ply398


In [46]:
from scipy.stats import ttest_rel

goals_per_match = (
    df
    .assign(
        FIRST_HALF = df["DURATION"] <= 45,
        SECOND_HALF = df["DURATION"] > 45
    )
    .groupby("MATCH_ID")
    .agg(
        goals_first_half=("FIRST_HALF", "sum"),
        goals_second_half=("SECOND_HALF", "sum")
    )
)


t_stat, p_value = ttest_rel(
    goals_per_match["goals_first_half"],
    goals_per_match["goals_second_half"]
)

t_stat, p_value



(np.float64(-5.492891644275833), np.float64(5.5353974071407444e-08))

In [40]:
alpha = 0.05

if p_value < alpha:
    print("Различие статистически значимо")
else:
    print("Нет оснований отвергнуть")


Различие статистически значимо


In [41]:
penalty_age = df[df["IS_PENALTY"] == 1]["PLAYER_AGE"]
non_penalty_age = df[df["IS_PENALTY"] == 0]["PLAYER_AGE"]

t_stat, p_value = stats.ttest_ind(penalty_age, non_penalty_age, nan_policy="omit")

t_stat, p_value


(np.float64(5.40542513865032), np.float64(7.173309697957723e-08))

In [42]:
if p_value < alpha:
    print("Пенальти чаще забивают более возрастные игроки")
else:
    print("Нет статистически значимого различия")


Пенальти чаще забивают более возрастные игроки


In [43]:
target = "TOTAL_MATCH_GOALS"

features = [
    "ATTENDANCE",
    "IS_PENALTY",
    "HOME_GOAL",
    "PLAYER_AGE",
    "FIRST_HALF_GOAL"
]

corr_df = df_encoded[features + [target]].corr()
corr_df[[target]]


,TOTAL_MATCH_GOALS
ATTENDANCE,0.056624
IS_PENALTY,-0.023038
HOME_GOAL,0.016896
PLAYER_AGE,0.012789
FIRST_HALF_GOAL,0.020284
TOTAL_MATCH_GOALS,1.000000
